### Movie Recommendation: Using Spark Mlib & Python DataFrame on Databricks Community cloud setup

The training dataset has been taken from http://files.grouplens.org/datasets/movielens/ml-1m.zip. This dataset has 1000209 ratings (userId::movieId::rating::timestamp), 3883 movies (movieId::title::genre1|genre2|genre3) & 6040 users (UserID::Gender::Age::Occupation::Zip-code).

This solution is based on DataBricks training material given in Scala. This solution uses Matrix Factorization based Collaborative Filtering using ALS (alternating least squares) optimization algorithm. This solution requirement is fulfilled by ALS estimator in Spark Mlib. The parametrized Grid Search approach has been deployed to identify best fit model.

In [2]:
%sh
rm -rf "/tmp/ml-1m"
rm -f "/tmp/ml-1m.zip"
curl -o /tmp/ml-1m.zip http://files.grouplens.org/datasets/movielens/ml-1m.zip
unzip /tmp/ml-1m.zip -d /tmp

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 15 5778k 15 893k 0 0 1513k 0 0:00:03 --:--:-- 0:00:03 1511k
100 5778k 100 5778k 0 0 8334k 0 --:--:-- --:--:-- --:--:-- 8326k
Archive: /tmp/ml-1m.zip
 creating: /tmp/ml-1m/
 inflating: /tmp/ml-1m/movies.dat 
 inflating: /tmp/ml-1m/ratings.dat 
 inflating: /tmp/ml-1m/README 
 inflating: /tmp/ml-1m/users.dat

In [3]:
%sh 
ls -lah /tmp

total 7.4M
drwxrwxrwt 1 root root 4.0K Oct 22 09:29 .
drwxr-xr-x 1 root root 4.0K Oct 22 07:01 ..
drwxr-xr-x 3 root root 4.0K Oct 22 07:22 1540192977667-0
drwx------ 2 root root 4.0K Oct 22 07:02 c1b5069e-8a86-4bd8-9f54-59fa362162dd_resources
-rw-r--r-- 1 root root 22 Oct 22 07:01 chauffeur-daemon-params
-rw-r--r-- 1 root root 5 Oct 22 07:01 chauffeur-daemon.pid
-rw-r--r-- 1 ubuntu ubuntu 156 Oct 22 07:01 chauffeur-env.sh
-rw-r--r-- 1 ubuntu ubuntu 133 Oct 22 07:01 custom-spark.conf
-rw-r--r-- 1 root root 19 Oct 22 07:01 driver-daemon-params
-rw-r--r-- 1 root root 5 Oct 22 07:01 driver-daemon.pid
-rw-r--r-- 1 root root 2.3K Oct 22 07:01 driver-env.sh
drwxr-xr-x 2 root root 4.0K Oct 22 07:02 hsperfdata_root
drwxrwxrwt 2 root root 4.0K Oct 22 07:01 .ICE-unix
srw------- 1 root root 0 Oct 22 07:02 .java_pid2016
-rw-r--r-- 1 root root 1.6M Oct 22 07:24 jniloader1433643340289574719netlib-native_system-linux-x86_64.so
-rw-r--r-- 1 root root 59K Oct 22 07:23 liblz4-java6319260522315353581.so
drwxr-x--- 2 root root 4.0K Jan 29 2016 ml-1m
-rw-r--r-- 1 root root 5.7M Oct 22 09:29 ml-1m.zip
drwx------ 4 root root 4.0K Oct 22 07:07 root
drwxr-xr-x 3 root root 4.0K Oct 22 07:02 Rserv
drwx------ 2 root root 4.0K Oct 22 07:02 RtmpIACTVr
-rw-r--r-- 1 root root 0 Oct 22 07:02 snowflake_jdbc0.log.0
-rw-r--r-- 1 root root 0 Oct 22 07:02 snowflake_jdbc0.log.0.lck
drwxr-xr-x 2 root root 4.0K Oct 22 07:02 spark-4b31d563-f32c-4c4d-9f0d-4a9ccb3c01ea
drwxr-xr-x 2 root root 4.0K Oct 22 07:02 spark-9098b46a-9273-45e8-a464-13e5f7160ee5
drwxr-xr-x 18 root root 4.0K Oct 22 07:23 spark-aa9e08a2-f6e8-44be-b3cb-a47dce11f53f
drwxr-xr-x 2 root root 4.0K Oct 22 07:02 spark-be9cbabe-2ca1-4c29-820b-477663f9f3ab
-rw-r--r-- 1 root root 4.3K Oct 22 07:02 temp1485612715403739502resource.r
-rw-r--r-- 1 root root 227 Oct 22 07:02 temp3626945699025447158resource.r
-rw-r--r-- 1 root root 1.4K Oct 22 07:02 temp8917640234118062402resource.r
-rw-r--r-- 1 root root 5.5K Oct 22 07:02 temp9139240647761794788resource.r
-rw------- 1 root root 0 Oct 22 07:22 tmp.armQQNnstz
drwxrwxrwt 2 root root 4.0K Oct 22 07:01 .X11-unix

In [4]:
%fs cp -r file:/tmp/ml-1m dbfs:/tmp/ml-1m

res2: Boolean = true

In [5]:
%fs ls /tmp/ml-1m/

path,name,size
dbfs:/tmp/ml-1m/README,README,5577
dbfs:/tmp/ml-1m/movies.dat,movies.dat,171308
dbfs:/tmp/ml-1m/ratings.dat,ratings.dat,24594131
dbfs:/tmp/ml-1m/users.dat,users.dat,134368


In [6]:
ratingsDF = sc.textFile("/tmp/ml-1m/ratings.dat") \
  .map(lambda l: l.split("::")) \
  .map(lambda l: (int(l[0]), int(l[1]), float(l[2]))) \
  .toDF(["user", "item", "label"])

ratingsDF.show(5, False)

+----+----+-----+
user|item|label|
+----+----+-----+
1 |1193|5.0 |
1 |661 |3.0 |
1 |914 |3.0 |
1 |3408|4.0 |
1 |2355|5.0 |
+----+----+-----+
only showing top 5 rows

In [7]:
moviesDF = sc.textFile("/tmp/ml-1m/movies.dat") \
  .map(lambda l: l.split("::")) \
  .map(lambda l: (int(l[0]), str(l[1]), l[2].split("|"))) \
  .toDF(["movie", "title", "genre"])

moviesDF.show(5, False)

+-----+----------------------------------+--------------------------------+
movie|title |genre |
+-----+----------------------------------+--------------------------------+
1 |Toy Story (1995) |[Animation, Children's, Comedy] |
2 |Jumanji (1995) |[Adventure, Children's, Fantasy]|
3 |Grumpier Old Men (1995) |[Comedy, Romance] |
4 |Waiting to Exhale (1995) |[Comedy, Drama] |
5 |Father of the Bride Part II (1995)|[Comedy] |
+-----+----------------------------------+--------------------------------+
only showing top 5 rows

In [8]:
(ratingsDF.schema, moviesDF.schema)

Out[ 40 ]: 
(StructType(List(StructField(user,LongType,true),StructField(item,LongType,true),StructField(label,DoubleType,true))),
 StructType(List(StructField(movie,LongType,true),StructField(title,StringType,true),StructField(genre,ArrayType(StringType,true),true))))

###Collaborative filtering
Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix, in our case, the user-movie rating matrix. MLlib currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors that can be used to predict missing entries. In particular, we implement the alternating least squares (ALS) algorithm to learn these latent factors.

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator

paramGrid = ParamGridBuilder() \
                    .addGrid(als.rank, [1, 5]) \
                    .addGrid(als.maxIter, [5, 10]) \
                    .addGrid(als.regParam, [0.3, 0.1, 0.01]) \
                    .addGrid(als.alpha, [2.0, 3.0]) \
                    .build()

als = ALS(userCol="user", itemCol="item", ratingCol="label", coldStartStrategy="drop")
cv = TrainValidationSplit(estimator=als, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, trainRatio=0.8)
model = cv.fit(ratingsDF)

In [11]:
best_model = model.bestModel
best_rank = best_model.rank
best_maxiter = (best_model._java_obj.parent().getMaxIter())

print("Best model's rank & maxiter:", best_rank, best_maxiter)

Best model's rank & maxiter: 10 10

In [12]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(model.transform(ratingsDF))
print("Root Mean Square Error (RMSE):", str(rmse))

Root Mean Square Error (RMSE): 0.8294707267212242

In [13]:
# Top 10 movies for users
userRecs = best_model.recommendForAllUsers(10)

# Top 10 users of all movies
movieRecs = best_model.recommendForAllItems(10)

In [14]:
# List first 5 users along with their top 10 recommended movies
userRecs.show(5, truncate = False)

+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
user|recommendations |
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
1580|[[557, 4.453588], [572, 4.2417707], [989, 4.2251987], [787, 4.125322], [1851, 4.098146], [2309, 4.0912337], [3338, 4.0475936], [2905, 4.0218296], [1423, 3.9921849], [53, 3.9904366]] |
4900|[[572, 5.9181323], [3233, 5.3571806], [3092, 5.3320293], [2197, 5.2342963], [527, 5.091989], [318, 5.0829754], [2324, 5.018231], [2999, 4.986596], [1851, 4.875985], [50, 4.859957]] |
5300|[[557, 5.399946], [2309, 5.2584367], [3338, 5.032308], [572, 5.031674], [787, 4.9906464], [989, 4.989652], [1423, 4.9369254], [53, 4.936117], [1002, 4.927404], [2503, 4.9026766]] |
471 |[[2905, 4.6471915], [3245, 4.627755], [2019, 4.5727324], [2309, 4.5691295], [670, 4.558665], [572, 4.5444427], [750, 4.5443335], [1851, 4.532837], [3338, 4.530421], [668, 4.5249753]] |
1591|[[572, 5.9330034], [557, 5.6049376], [989, 5.407812], [787, 5.355318], [2503, 5.3154287], [3233, 5.2779946], [2905, 5.2425466], [3172, 5.226792], [2019, 5.2161746], [3517, 5.2138357]]|
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [15]:
# List first 5 movies along with their top 10 recommended-to-users
movieRecs.show(5, truncate = False)

+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
item|recommendations |
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
1580|[[3902, 5.055544], [283, 5.0490103], [2867, 4.8793364], [5862, 4.8286247], [101, 4.7790847], [1673, 4.7630944], [4801, 4.7393966], [4086, 4.733805], [5072, 4.709725], [1856, 4.7073]] |
471 |[[283, 4.6444526], [1746, 4.593036], [2694, 4.5929523], [2775, 4.509493], [53, 4.501582], [597, 4.4882317], [878, 4.4789944], [5137, 4.4769454], [381, 4.4765177], [882, 4.4465184]] |
1591|[[3902, 4.065948], [283, 3.9715333], [2867, 3.9586709], [2155, 3.945363], [1856, 3.8070617], [4801, 3.802698], [5670, 3.7410824], [356, 3.7181637], [1673, 3.676835], [3834, 3.655352]]|
1342|[[46, 4.682841], [2155, 4.5519185], [3844, 4.33701], [5729, 4.2950077], [4801, 4.2001047], [5670, 4.1499877], [283, 4.142176], [2694, 4.121922], [3902, 4.1100597], [1186, 4.029523]] |
2122|[[46, 4.795556], [2155, 4.7693696], [3844, 4.5330424], [5670, 4.48994], [4801, 4.2801514], [5099, 4.2765365], [3915, 4.259888], [3801, 4.163019], [3902, 4.158675], [5729, 4.1247125]] |
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows